In [759]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
import category_encoders as ce
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score  
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [760]:
data=pd.read_fwf('data-01')
# print(data)
data[['Date','Time','Category','Target']] = data['Date\tTime\tCategory\tTarget'].str.split('\t',expand=True)
data.drop(columns=['Date\tTime\tCategory\tTarget'], inplace = True)
# data
# data.rename(columns = {'27-03-1988':'Date'}, inplace = True)
# data.rename(columns = {'08:00:00':'Time'}, inplace = True)
# data.rename(columns = {'58':'Category'}, inplace = True)
# data.rename(columns = {'134':'Glucose_Score'}, inplace = True)

# data.to_csv('data.csv', index = None)
# # print(data)
# data=pd.read_csv('data.csv')
data

,Date,Time,Category,Target
0,27-03-1988,08:00:00,58,134
1,27-03-1988,08:00:00,34,20
2,27-03-1988,12:00:00,60,158
3,27-03-1988,18:00:00,62,258
4,28-03-1988,08:00:00,58,115
...,...,...,...,...
395,27-07-1988,08:00:00,58,139
396,27-07-1988,08:00:00,34,12
397,27-07-1988,12:00:00,60,107
398,27-07-1988,18:00:00,62,102


In [761]:
# Example DataFrame
# df = pd.DataFrame({'Time':['02:45', '10:30', '18:20']})

# Define a function to split the time string and convert to integers
def split_time(time_str):
    hours_str, minutes_str, seconds_str = time_str.split(':')
    hours = (hours_str)
    minutes = (minutes_str)
    seconds = (seconds_str)
    return hours, minutes, seconds

# Apply the function to the Time column of the DataFrame
data[['Hours', 'Minutes','Seconds']] = data['Time'].apply(lambda x: pd.Series(split_time(x)))

# Print the result
print(data)

           Date      Time Category Target Hours Minutes Seconds
0    27-03-1988  08:00:00       58    134    08      00      00
1    27-03-1988  08:00:00       34     20    08      00      00
2    27-03-1988  12:00:00       60    158    12      00      00
3    27-03-1988  18:00:00       62    258    18      00      00
4    28-03-1988  08:00:00       58    115    08      00      00
..          ...       ...      ...    ...   ...     ...     ...
395  27-07-1988  08:00:00       58    139    08      00      00
396  27-07-1988  08:00:00       34     12    08      00      00
397  27-07-1988  12:00:00       60    107    12      00      00
398  27-07-1988  18:00:00       62    102    18      00      00
399  28-07-1988  08:00:00       58     73    08      00      00

[400 rows x 7 columns]


In [762]:
# concatenating the DataFrames
det = pd.concat([data['Hours'], data['Minutes']], join = 'outer', axis = 1)

# displaying the DataFrame
# print(det)
data=data.drop(['Date','Time'], axis=1)
print(data)
data['time'] = data['Hours'].astype(str) + data['Minutes'].astype(str)
data['target'] = data['Target']
data=data.drop(['Hours','Minutes','Seconds','Target'], axis=1)
print(data)


    Category Target Hours Minutes Seconds
0         58    134    08      00      00
1         34     20    08      00      00
2         60    158    12      00      00
3         62    258    18      00      00
4         58    115    08      00      00
..       ...    ...   ...     ...     ...
395       58    139    08      00      00
396       34     12    08      00      00
397       60    107    12      00      00
398       62    102    18      00      00
399       58     73    08      00      00

[400 rows x 5 columns]
    Category  time target
0         58  0800    134
1         34  0800     20
2         60  1200    158
3         62  1800    258
4         58  0800    115
..       ...   ...    ...
395       58  0800    139
396       34  0800     12
397       60  1200    107
398       62  1800    102
399       58  0800     73

[400 rows x 3 columns]


In [773]:
X = data.iloc[:, 0:-1].values
y = data.iloc[:, -1].values
y = y.reshape(-1,1)
print(X)
print(y)
X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.2,random_state=42)
X_train
y_train


[['58' '0800']
 ['34' '0800']
 ['60' '1200']
 ['62' '1800']
 ['58' '0800']
 ['34' '0800']
 ['60' '1200']
 ['62' '1800']
 ['58' '0800']
 ['34' '0800']
 ['60' '1200']
 ['62' '1800']
 ['58' '0800']
 ['34' '0800']
 ['60' '1200']
 ['62' '1800']
 ['58' '0800']
 ['34' '0800']
 ['62' '1800']
 ['58' '0800']
 ['34' '0800']
 ['60' '1200']
 ['62' '1800']
 ['58' '0800']
 ['34' '0800']
 ['60' '1200']
 ['62' '1800']
 ['58' '0800']
 ['34' '0800']
 ['60' '1200']
 ['62' '1800']
 ['58' '0800']
 ['34' '0800']
 ['60' '1200']
 ['62' '1800']
 ['58' '0800']
 ['34' '0800']
 ['60' '1200']
 ['62' '1800']
 ['58' '0800']
 ['34' '0800']
 ['60' '1200']
 ['62' '1800']
 ['58' '0800']
 ['34' '0800']
 ['60' '1200']
 ['62' '1800']
 ['58' '0800']
 ['34' '0800']
 ['60' '1200']
 ['62' '1800']
 ['58' '0800']
 ['34' '0800']
 ['60' '1200']
 ['62' '1800']
 ['58' '0800']
 ['34' '0800']
 ['60' '1200']
 ['62' '1800']
 ['58' '0800']
 ['34' '0800']
 ['60' '1200']
 ['62' '1800']
 ['58' '0800']
 ['34' '0800']
 ['60' '1200']
 ['62' '18

array([['258'],
       ['105'],
       ['179'],
       ['159'],
       ['118'],
       ['115'],
       ['12'],
       ['26'],
       ['20'],
       ['131'],
       ['100'],
       ['97'],
       ['20'],
       ['12'],
       ['15'],
       ['75'],
       ['22'],
       ['164'],
       ['127'],
       ['111'],
       ['129'],
       ['178'],
       ['118'],
       ['111'],
       ['104'],
       ['135'],
       ['148'],
       ['53'],
       ['26'],
       ['147'],
       ['91'],
       ['12'],
       ['116'],
       ['113'],
       ['12'],
       ['20'],
       ['110'],
       ['12'],
       ['112'],
       ['12'],
       ['44'],
       ['20'],
       ['20'],
       ['134'],
       ['13'],
       ['88'],
       ['72'],
       ['120'],
       ['75'],
       ['66'],
       ['110'],
       ['61'],
       ['140'],
       ['125'],
       ['93'],
       ['114'],
       ['94'],
       ['155'],
       ['131'],
       ['162'],
       ['20'],
       ['74'],
       ['130'],
       ['65'],
       

In [764]:
# ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [2])], remainder='passthrough')
# X_train = np.array(ct.fit_transform(X_train))
# X_train

In [765]:
encoder =ce.BinaryEncoder(cols=[0])
print(X_train)
X_train=np.array(encoder.fit_transform(X_train))
X_train
X_test=np.array(encoder.fit_transform(X_test))


[['62' '1800']
 ['62' '1800']
 ['62' '1800']
 ['62' '1800']
 ['60' '1200']
 ['58' '0800']
 ['34' '0800']
 ['34' '0800']
 ['34' '0800']
 ['62' '1800']
 ['60' '1200']
 ['48' '2200']
 ['34' '0800']
 ['34' '0800']
 ['34' '0800']
 ['60' '1200']
 ['34' '0800']
 ['62' '1800']
 ['58' '0800']
 ['58' '0800']
 ['58' '0800']
 ['62' '1800']
 ['58' '0800']
 ['62' '1800']
 ['60' '1200']
 ['62' '1800']
 ['58' '0800']
 ['60' '1200']
 ['34' '0800']
 ['60' '1200']
 ['58' '0800']
 ['34' '0800']
 ['58' '0800']
 ['58' '0800']
 ['34' '0800']
 ['34' '0800']
 ['58' '0800']
 ['34' '0800']
 ['60' '1200']
 ['34' '0800']
 ['60' '1200']
 ['34' '0800']
 ['34' '0800']
 ['58' '0800']
 ['34' '0800']
 ['58' '0800']
 ['62' '1800']
 ['60' '1200']
 ['58' '0800']
 ['60' '1200']
 ['58' '0800']
 ['60' '1200']
 ['62' '1800']
 ['60' '1200']
 ['60' '1200']
 ['62' '1800']
 ['60' '1200']
 ['58' '0800']
 ['62' '1800']
 ['62' '1800']
 ['34' '0800']
 ['58' '0800']
 ['62' '1800']
 ['58' '0800']
 ['34' '0800']
 ['58' '0800']
 ['58' '08

In [766]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)
y_train = sc.fit_transform(y_train)
y_test = sc.fit_transform(y_test)
print(X_train)
print(X_test)

[[-0.61105692 -0.96922337  0.93345864  1.3562472 ]
 [-0.61105692 -0.96922337  0.93345864  1.3562472 ]
 [-0.61105692 -0.96922337  0.93345864  1.3562472 ]
 ...
 [-0.61105692  1.03175391 -1.07128475  0.0249616 ]
 [ 1.63650876 -0.96922337 -1.07128475 -0.86256212]
 [-0.61105692 -0.96922337  0.93345864  1.3562472 ]]
[[-0.71374643 -0.85972695  1.          0.04957418]
 [-0.71374643  1.16316    -1.         -0.83174449]
 [-0.71374643 -0.85972695  1.          0.04957418]
 [-0.71374643  1.16316     1.          1.37155218]
 [-0.71374643 -0.85972695  1.          0.04957418]
 [ 1.4010578  -0.85972695 -1.         -0.83174449]
 [-0.71374643  1.16316    -1.         -0.83174449]
 [-0.71374643  1.16316     1.          1.37155218]
 [ 1.4010578  -0.85972695 -1.         -0.83174449]
 [-0.71374643  1.16316    -1.         -0.83174449]
 [ 1.4010578  -0.85972695 -1.         -0.83174449]
 [-0.71374643  1.16316     1.          1.37155218]
 [ 1.4010578  -0.85972695 -1.         -0.83174449]
 [ 1.4010578  -0.85972695

In [767]:
# from sklearn.svm import SVR
# regressor = SVR(kernel = 'linear')
# regressor.fit(X_train, y_train)
# y_pred=regressor.predict(X_test)

In [768]:
# mse = mean_squared_error(y_test, y_pred)
# rmse = np.sqrt(mse)
# mae = mean_absolute_error(y_test, y_pred)
# r2 = r2_score(y_test, y_pred)

# # Print the evaluation metrics
# print('MSE:', mse)
# print('RMSE:', rmse)
# print('MAE:', mae)
# print('R2 score:', r2)

In [769]:
from sklearn.svm import SVR
regressor = SVR(kernel = 'rbf')
regressor.fit(X_train, y_train)
y_pred_poly = regressor.predict(X_test)
mse = mean_squared_error(y_test, y_pred_poly)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred_poly)
r2 = r2_score(y_test, y_pred_poly)
print('MSE:', mse)
print('RMSE:', rmse)
print('MAE:', mae)
print('R2 score:', r2)

MSE: 0.39035542780227833
RMSE: 0.6247843050223639
MAE: 0.3825269370698387
R2 score: 0.6096445721977217


c:\Users\ayush\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [770]:
# from sklearn.ensemble import RandomForestRegressor
# regressor = RandomForestRegressor(n_estimators = 10, random_state = 0)
# regressor.fit(X_train, y_train)
# y_pred_rf = regressor.predict(X_test)
# mse = mean_squared_error(y_test, y_pred_rf)
# rmse = np.sqrt(mse)
# mae = mean_absolute_error(y_test, y_pred_rf)
# r2 = r2_score(y_test, y_pred_rf)

# # Print the evaluation metrics
# print('MSE:', mse)
# print('RMSE:', rmse)
# print('MAE:', mae)
# print('R2 score:', r2)

In [771]:
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import PassiveAggressiveRegressor

# Initialize the SGDRegressor with desired hyperparameters
# regressor = SGDRegressor(loss='squared_error', penalty='l2', max_iter=1000, tol=1e-3)
regressor=PassiveAggressiveRegressor()
# Train the model with initial data using fit method
# X_initial, y_initial = load_initial_data()  # Load initial data
regressor.fit(X_train, y_train)  # Train the model with initial data
y_pred_sgd=regressor.predict(X_test)
mse = mean_squared_error(y_test, y_pred_poly)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred_poly)
r2 = r2_score(y_test, y_pred_poly)
print('MSE:', mse)
print('RMSE:', rmse)
print('MAE:', mae)
print('R2 score:', r2)

# Continue learning with new data using partial_fit method
# X_new, y_new = load_new_data()  # Load new data
# regressor.partial_fit(X_new, y_new)  # Update the model with new data

# # Perform predictions on new data
# X_test, y_test = load_test_data()  # Load test data
# y_pred = regressor.predict(X_test)  # Perform predictions on test data


MSE: 0.39035542780227833
RMSE: 0.6247843050223639
MAE: 0.3825269370698387
R2 score: 0.6096445721977217


c:\Users\ayush\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [772]:
for i in range(2,70):
    data=pd.read_fwf('data-01')
    # print(data)
    data[['Date','Time','Category','Target']] = data['Date\tTime\tCategory\tTarget'].str.split('\t',expand=True)
    data.drop(columns=['Date\tTime\tCategory\tTarget'], inplace = True)
    def split_time(time_str):
        hours_str, minutes_str, seconds_str = time_str.split(':')
        hours = (hours_str)
        minutes = (minutes_str)
        seconds = (seconds_str)
    
        return hours, minutes, seconds
    #Apply the function to the Time column of the DataFrame
    data[['Hours', 'Minutes','Seconds']] = data['Time'].apply(lambda x: pd.Series(split_time(x)))
    det = pd.concat([data['Hours'], data['Minutes']], join = 'outer', axis = 1)
    # displaying the DataFrame
    # print(det)
    data=data.drop(['Date','Time'], axis=1)
    print(data)
    data['time'] = data['Hours'].astype(str) + data['Minutes'].astype(str)
    data['target'] = data['Target']
    data=data.drop(['Hours','Minutes','Seconds','Target'], axis=1)
    print(data)
    X_new = data.iloc[:, 0:-1].values
    y_new = data.iloc[:, -1].values
    y_new = y_new.reshape(-1,1)
    print(X_new)
    print(y_new)
    encoder =ce.BinaryEncoder(cols=[0])
    print(X_new)
    X_new=np.array(encoder.fit_transform(X_new))
    X_new = sc.fit_transform(X_new)
    y_new = sc.fit_transform(y_new)
    regressor.partial_fit(X_new, y_new)
    
    y_pred=regressor.predict(X_new)

    # mse = mean_squared_error(y_test, y_pred)
    # rmse = np.sqrt(mse)
    # mae = mean_absolute_error(y_test, y_pred)
    # r2 = r2_score(y_test, y_pred)
    # print('MSE:', mse)
    # print('RMSE:', rmse)
    # print('MAE:', mae)
    # print('R2 score:', r2)
    y_pred=y_pred.reshape(-1,1)
    # Reverse the scaling of predicted values
    y_pred = sc.inverse_transform(y_pred)

    # Note: y_pred_scaled is the predicted values obtained from the model in the scaled form, and y_pred_unscaled will contain the predicted values in the original unscaled form

    print(y_pred)


    





    Category Target Hours Minutes Seconds
0         58    134    08      00      00
1         34     20    08      00      00
2         60    158    12      00      00
3         62    258    18      00      00
4         58    115    08      00      00
..       ...    ...   ...     ...     ...
395       58    139    08      00      00
396       34     12    08      00      00
397       60    107    12      00      00
398       62    102    18      00      00
399       58     73    08      00      00

[400 rows x 5 columns]
    Category  time target
0         58  0800    134
1         34  0800     20
2         60  1200    158
3         62  1800    258
4         58  0800    115
..       ...   ...    ...
395       58  0800    139
396       34  0800     12
397       60  1200    107
398       62  1800    102
399       58  0800     73

[400 rows x 3 columns]
[['58' '0800']
 ['34' '0800']
 ['60' '1200']
 ['62' '1800']
 ['58' '0800']
 ['34' '0800']
 ['60' '1200']
 ['62' '1800']
 ['58' '0800']
 

c:\Users\ayush\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\ayush\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



    Category  time target
0         58  0800    134
1         34  0800     20
2         60  1200    158
3         62  1800    258
4         58  0800    115
..       ...   ...    ...
395       58  0800    139
396       34  0800     12
397       60  1200    107
398       62  1800    102
399       58  0800     73

[400 rows x 3 columns]
[['58' '0800']
 ['34' '0800']
 ['60' '1200']
 ['62' '1800']
 ['58' '0800']
 ['34' '0800']
 ['60' '1200']
 ['62' '1800']
 ['58' '0800']
 ['34' '0800']
 ['60' '1200']
 ['62' '1800']
 ['58' '0800']
 ['34' '0800']
 ['60' '1200']
 ['62' '1800']
 ['58' '0800']
 ['34' '0800']
 ['62' '1800']
 ['58' '0800']
 ['34' '0800']
 ['60' '1200']
 ['62' '1800']
 ['58' '0800']
 ['34' '0800']
 ['60' '1200']
 ['62' '1800']
 ['58' '0800']
 ['34' '0800']
 ['60' '1200']
 ['62' '1800']
 ['58' '0800']
 ['34' '0800']
 ['60' '1200']
 ['62' '1800']
 ['58' '0800']
 ['34' '0800']
 ['60' '1200']
 ['62' '1800']
 ['58' '0800']
 ['34' '0800']
 ['60' '1200']
 ['62' '1800']
 ['58' '0800']
 ['

c:\Users\ayush\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\ayush\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\ayush\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\ayush\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:1143: DataC

[[189.19145177]
 [ 46.49941257]
 [172.07322877]
 [ 50.17723347]
 [189.19145177]
 [ 46.49941257]
 [172.07322877]
 [ 50.17723347]
 [189.19145177]
 [ 46.49941257]
 [172.07322877]
 [ 50.17723347]
 [189.19145177]
 [ 46.49941257]
 [172.07322877]
 [ 50.17723347]
 [189.19145177]
 [ 46.49941257]
 [ 50.17723347]
 [189.19145177]
 [ 46.49941257]
 [172.07322877]
 [ 50.17723347]
 [189.19145177]
 [ 46.49941257]
 [172.07322877]
 [ 50.17723347]
 [189.19145177]
 [ 46.49941257]
 [172.07322877]
 [ 50.17723347]
 [189.19145177]
 [ 46.49941257]
 [172.07322877]
 [ 50.17723347]
 [189.19145177]
 [ 46.49941257]
 [172.07322877]
 [ 50.17723347]
 [189.19145177]
 [ 46.49941257]
 [172.07322877]
 [ 50.17723347]
 [189.19145177]
 [ 46.49941257]
 [172.07322877]
 [ 50.17723347]
 [189.19145177]
 [ 46.49941257]
 [172.07322877]
 [ 50.17723347]
 [189.19145177]
 [ 46.49941257]
 [172.07322877]
 [ 50.17723347]
 [189.19145177]
 [ 46.49941257]
 [172.07322877]
 [ 50.17723347]
 [189.19145177]
 [ 46.49941257]
 [172.07322877]
 [ 50.17

...

c:\Users\ayush\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\ayush\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[[ 86.51432828]
 [  6.07746416]
 [ 77.28651556]
 [189.46557243]
 [ 86.51432828]
 [  6.07746416]
 [ 77.28651556]
 [189.46557243]
 [ 86.51432828]
 [  6.07746416]
 [ 77.28651556]
 [189.46557243]
 [ 86.51432828]
 [  6.07746416]
 [ 77.28651556]
 [189.46557243]
 [ 86.51432828]
 [  6.07746416]
 [189.46557243]
 [ 86.51432828]
 [  6.07746416]
 [ 77.28651556]
 [189.46557243]
 [ 86.51432828]
 [  6.07746416]
 [ 77.28651556]
 [189.46557243]
 [ 86.51432828]
 [  6.07746416]
 [ 77.28651556]
 [189.46557243]
 [ 86.51432828]
 [  6.07746416]
 [ 77.28651556]
 [189.46557243]
 [ 86.51432828]
 [  6.07746416]
 [ 77.28651556]
 [189.46557243]
 [ 86.51432828]
 [  6.07746416]
 [ 77.28651556]
 [189.46557243]
 [ 86.51432828]
 [  6.07746416]
 [ 77.28651556]
 [189.46557243]
 [ 86.51432828]
 [  6.07746416]
 [ 77.28651556]
 [189.46557243]
 [ 86.51432828]
 [  6.07746416]
 [ 77.28651556]
 [189.46557243]
 [ 86.51432828]
 [  6.07746416]
 [ 77.28651556]
 [189.46557243]
 [ 86.51432828]
 [  6.07746416]
 [ 77.28651556]
 [189.46

c:\Users\ayush\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\ayush\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[[118.28329728]
 [ 16.28811287]
 [101.19660852]
 [139.98894756]
 [118.28329728]
 [ 16.28811287]
 [101.19660852]
 [139.98894756]
 [118.28329728]
 [ 16.28811287]
 [101.19660852]
 [139.98894756]
 [118.28329728]
 [ 16.28811287]
 [101.19660852]
 [139.98894756]
 [118.28329728]
 [ 16.28811287]
 [139.98894756]
 [118.28329728]
 [ 16.28811287]
 [101.19660852]
 [139.98894756]
 [118.28329728]
 [ 16.28811287]
 [101.19660852]
 [139.98894756]
 [118.28329728]
 [ 16.28811287]
 [101.19660852]
 [139.98894756]
 [118.28329728]
 [ 16.28811287]
 [101.19660852]
 [139.98894756]
 [118.28329728]
 [ 16.28811287]
 [101.19660852]
 [139.98894756]
 [118.28329728]
 [ 16.28811287]
 [101.19660852]
 [139.98894756]
 [118.28329728]
 [ 16.28811287]
 [101.19660852]
 [139.98894756]
 [118.28329728]
 [ 16.28811287]
 [101.19660852]
 [139.98894756]
 [118.28329728]
 [ 16.28811287]
 [101.19660852]
 [139.98894756]
 [118.28329728]
 [ 16.28811287]
 [101.19660852]
 [139.98894756]
 [118.28329728]
 [ 16.28811287]
 [101.19660852]
 [139.98

c:\Users\ayush\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
